In [21]:
import scipy.stats as sts
import pandas as pd
import numpy as np
from statsmodels.stats.weightstats import _tconfint_generic

### В этом блоке проверяем гипотезу о том, что по утрам наблюдается всплеск в спреде. 

Начнем с того что просто посмотрим на то как выглядят спреды разных бирж в зависимости от времени.
Можно легко заметить что в начале дня в промежутке между 6 и 7 утра визуально наблюдается всплеск в спредах.
По всей видимости именно он и является причиной поставленного эксперимента.

Также мы знаем что среднее значение спреда является положительным, а значит мы должны иметь распределение со средним находящимся справа от нуля. Также следует отметить что невозможна ситуация при которой дельта в спреде меньше 0, а значит мы работаем с распределением ограниченным нулем слева. Интерес представляет посмотреть на форму распределений у спредов в течение дня для различных бирж. Для того чтобы визуализировать распределения стоит построить гистограммы.

Так как видно, что средние значения спредов отличаются для каждого из рынков, весьма логичным действием является осуществление первичной предобработки данных. Для этого воспользуемся стандартным обработчиком из sklearn  

Построим аналог BBands, где в качестве верхнего и нижнего уровня будут выступать границы доверительных интервалов. 
Таким образом нам удастся посмотреть на динамику изменения доверительных интервалов на всем временном промежутке.

Тут важно заметить что мы не знаем теоретическую дисперсию $\sigma_{C, B, ...}}$, поэтому корректней будет использовать t-интервал (использование выборочных дисперсий). Для такого случая доверительные интервалы примут вид


$$\hat{X_t} \pm t_{1 - \frac{\alpha}{2}}\frac{S}{\sqrt{n}}$$.

Чтобы самому это не писать, используем statsmodels

In [50]:
def create_bb_analog(observations: pd.Series, window_size_sec: int = 10, alpha=0.05):
    def apply_interval_to_window(ser: pd.Series):
        mean_value = ser.mean()
        std_value = ser.std(ddof=1)/np.sqrt(len(ser))
        return  _tconfint_generic(mean_value, std_value, len(ser) - 1, alpha, 'two-sided')
    intervals = pd.DataFrame([apply_interval_to_window(window) for window in observations.rolling(f'{window_size_sec}s')])
    intervals.columns = ["interval_left", "interval_right"]
    intervals.index = observations.index
    return pd.concat([observations, intervals], axis=1)



test_NUM = 40
test_series = pd.Series([1] * test_NUM, index=[pd.Timestamp(f'20130101 09:00:{_ if _ >=10 else f"0{_}"}') for _ in range(test_NUM)])
create_bb_analog(observations=test_series, window_size_sec=3)

,0,interval_left,interval_right
2013-01-01 09:00:00,1,NaN,NaN
2013-01-01 09:00:01,1,1.0,1.0
2013-01-01 09:00:02,1,1.0,1.0
2013-01-01 09:00:03,1,1.0,1.0
2013-01-01 09:00:04,1,1.0,1.0
2013-01-01 09:00:05,1,1.0,1.0
2013-01-01 09:00:06,1,1.0,1.0
2013-01-01 09:00:07,1,1.0,1.0
2013-01-01 09:00:08,1,1.0,1.0
2013-01-01 09:00:09,1,1.0,1.0


## Проверка на нормальность распределения разницы между максимальными и минимальными ценами